In [11]:
!pip install nltk
!pip install rouge-score
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 5.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.1/498.1 kB 5.6 MB/s eta 0:00:000:00:01m eta 0:00:01


In [4]:
import os
import csv
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from evaluate import load
bertscore = load("bertscore")
class EvaluateModel(object):
    
    def __init__(self):
        print('Evaluating Results...')
        self.path = "."#os.getcwd()+'/../../Data'
        self.categories = ['purpose']#['purpose','description','scope','requirements','assumptions and dependencies', 'architecture','users']
        self.evalData = self.path + "/evaluation_data.csv"
        self.df = None
        self.readEvaluation()
        #self.getBleuScore()
        self.scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
        self.getRougeScore()
        self.getBertScore()
        self.writeResults()
        
    
    def tokenize(self, column):
        column = column.apply(lambda x: x.split())
        return column
    
    def readEvaluation(self):
        
        self.df = pd.read_csv(self.evalData, encoding = "latin-1")
        self.df.fillna("", inplace = True)
    
    def getBleuScore(self):
        df = self.df.copy()
        df.predicted = self.tokenize(df.predicted)
        df.expected = self.tokenize(df.expected)
        scores = []
        
    
        for index, row in df.iterrows():
            
            bleu_score = sentence_bleu([row.expected], row.predicted)
            scores.append(bleu_score)
        
        self.df["bleu_score"] = scores
    
    def getRougeScore(self):
        df = self.df.copy()
        rPrec = []
        rRec = []
        rF = []
        
        for index, row in df.iterrows():
            rouge_score = self.scorer.score(row.expected, row.predicted)
            rPrec.append(rouge_score['rouge1'].precision)
            rRec.append(rouge_score['rouge1'].recall)
            rF.append(rouge_score['rouge1'].fmeasure)
        
        self.df["ROUGE_precision"] = rPrec
        self.df["ROUGE_recall"] = rRec
        self.df["ROUGE_fmeasure"] = rF
    
    def getBertScore(self):
        df = self.df.copy()
        rPrec = []
        rRec = []
        rF = []
        
        for index, row in df.iterrows():
            score = bertscore.compute(predictions=[row.predicted], references=[row.expected], lang="en")
            rPrec.append(score['precision'][0])
            rRec.append(score['recall'][0])
            rF.append(score['f1'][0])
        self.df["BERTScore_precision"] = rPrec
        self.df["BERTScore_recall"] = rRec
        self.df["BERTScore_f1"] = rF
        

    
    def writeResults(self):
        self.df.to_csv(self.path + "/evaluated_data3.csv")
        del self.df["predicted"]
        del self.df["expected"]
        del self.df["doc"]
        print(list(self.df["ROUGE_recall"]))
        self.df.insert(0, "Categories", ['purpose','description','scope','requirements','assumptions and dependencies','architecture','users'])
        self.df["ROUGE_recall"] = []
        display(self.df)
    
EvaluateModel()

Evaluating Results...
[0.4090909090909091, 0.3333333333333333, 0.7455621301775148, 0.3633720930232558, 0.7121212121212122, 0.5235109717868338, 0.9572649572649573]


ValueError: Length of values (0) does not match length of index (7)

In [37]:
from evaluate import load
bertscore = load("bertscore")
results = bertscore.compute(predictions=["This section will cover general information about the project perspective,  functions, and various requirements and constraints. More detailed descriptions of the project requirements and functions will be covered in latter sections. 2.1 Product Perspective  This project is designed to be used by employees of several automotive plants to Prototype V1 will have little functionality, and its main purpose will be to display  the user interface as well as to show our interpretation of the requirements. 5.2 Sample Scenarios  An employee, Bob, was given the unique four-digit-code 7924 by their The system is a desktop application that requires a computer running Windows  Vista SP2 or higher version of Windows operating system. The computer must have Visual C++ Redistributable 2010, as well as .NET Framework 4.0 installed.  Prototype V1 will have little functionality, and its main purpose will be to display Vista SP2 or higher version of Windows operating system. The computer must have Visual C++ Redistributable 2010, as well as .NET Framework 4.0 installed.  Prototype V1 will have little functionality, and its main purpose will be to display  the user interface as well as to show our interpretation of the requirements. 5.2 Sample Scenarios Specification document will further explain the specifics of the software system being produced. Section 2 focuses on the context for needing the proposed system, how it should function, and the related constraints. Section 3 provides a detailed listing of the requirements specific to the system following a hierarchical numbering scheme. Section 4 addresses the modeling requirements which includes use cases diagrams, a state diagram, and a high-level class diagram. Section 5 discusses the proposed prototype and the details of the system with instructions on how to run it as well as sample scenarios. Section 6 contains a list of references used throughout the document. Finally, Section 7 provides contact information used for further inquiries. 2 Overall Description  This section will cover general information about the project perspective,  functions, and various requirements and constraints. More detailed descriptions of the project requirements and functions will be covered in latter sections. 2.1 Product Perspective "], references=["Project Automate Paint Defect Analysis Solution to the client’s need to automate the recording of paint defects on vehicles during production."], lang="en")

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

In [38]:
results

{'precision': [0.75494384765625],
 'recall': [0.8310678005218506],
 'f1': [0.7911789417266846],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.24.0)'}